In [1]:
import pandas as pd, numpy as np
from API_calls_functions import *
from Analysis_functions import *

In [2]:
OTP_path = 'OTP_data_W48-W5.csv' #'OTP_data_W1-W4.csv'
CC_path = 'CC_data_Dec_Jan.csv'
Spend_path = 'Address Correction Spend.csv'

Read Data

In [3]:
OTP = pd.read_csv(OTP_path)
Nb_weeks = OTP['HELLOFRESH_WEEK'].nunique()
Spend = pd.read_csv(Spend_path)
CC = pd.read_csv(CC_path)
print("Nb weeks",Nb_weeks)

Nb weeks 10


Define Functions

In [19]:
def get_box_type(du_id):
    if du_id.endswith('R'):
        return 'Remake'
    elif du_id.endswith('V'):
        return 'VIP'
    else:
        return 'Original'

def get_OTP_Counts(OTP):
    grouped_counts = OTP.groupby(['OTP_GROUP']).size().reset_index(name='total count')
    grouped_counts['perc'] = round(grouped_counts['total count'].transform(lambda x: x / x.sum() * 100),2)
    grouped_counts['weekly count'] = round(grouped_counts['total count'] / Nb_weeks)
    return grouped_counts

def retrieve_OTP_weekly_counts(OTP):
    non_flags_OTP_perc=dict()
    non_flags_OTP_perc['L1']=OTP.loc[OTP['OTP_GROUP'] == 'Late_1', 'weekly count'].values[0]
    non_flags_OTP_perc['L2']=OTP.loc[OTP['OTP_GROUP'] == 'Late_2', 'weekly count'].values[0]
    non_flags_OTP_perc['U']=OTP.loc[OTP['OTP_GROUP'] == 'Undelivered', 'weekly count'].values[0]
    return non_flags_OTP_perc

def retrieve_OTP_perc(OTP):
    non_flags_OTP_perc=dict()
    non_flags_OTP_perc['L1']=round(OTP.loc[OTP['OTP_GROUP'] == 'Late_1', 'perc'].values[0]/100,3)
    non_flags_OTP_perc['L2']=round(OTP.loc[OTP['OTP_GROUP'] == 'Late_2', 'perc'].values[0]/100,3)
    non_flags_OTP_perc['U']=round(OTP.loc[OTP['OTP_GROUP'] == 'Undelivered', 'perc'].values[0]/100,3)
    return non_flags_OTP_perc

def retrieve_avg_comp(address_flags_CC_costs_OTP):
    avg_comp=dict()
    avg_comp['L1']=address_flags_CC_costs_OTP.loc[address_flags_CC_costs_OTP['OTP_GROUP'] == 'Late_1', 'avg box comp'].values[0]
    avg_comp['L2']=address_flags_CC_costs_OTP.loc[address_flags_CC_costs_OTP['OTP_GROUP'] == 'Late_2', 'avg box comp'].values[0]
    avg_comp['U']=address_flags_CC_costs_OTP.loc[address_flags_CC_costs_OTP['OTP_GROUP'] == 'Undelivered', 'avg box comp'].values[0]
    return avg_comp

def compute_savings_per_OTP(flags_OTP_count,flags_OTP_perc,non_flags_OTP_perc,avg_comp):
    res=dict()
    for key in avg_comp.keys():
        res[key]=dict()
        new_OTP_nb=flags_OTP_count[key]/flags_OTP_perc[key]*non_flags_OTP_perc[key]
        gap=flags_OTP_count[key]-new_OTP_nb
        comp_savings=gap*avg_comp[key]
        res[key]['gap']=round(gap,2)
        res[key]['savings']=round(comp_savings,2)
    return res

def get_carrier_charges(OTP):
    grouped_counts = OTP.groupby(['brand']).agg({'correction_charge':'sum'}).reset_index()
    grouped_counts['weekly charges'] = round(grouped_counts['correction_charge'] / Nb_weeks)
    return grouped_counts

def get_positive_carrier_charges_and_OTP(OTP):
    OTP_positive_charges=OTP[OTP['correction_charge'] > 0]
    grouped_counts = OTP_positive_charges.groupby(['OTP_GROUP']).agg({'correction_charge': 'sum', 'BOX_ID': 'count'}).reset_index()
    grouped_counts['weekly charges'] = round(grouped_counts['correction_charge'] / Nb_weeks)
    grouped_counts['weekly count'] = round(grouped_counts['BOX_ID'] / Nb_weeks)
    grouped_counts['count perc'] = round(grouped_counts['weekly count'].transform(lambda x: x / x.sum() * 100),2)
    return grouped_counts

def get_CC_for_corrected_boxes(OTP,CC):
    box_with_positive_charges=OTP[OTP['correction_charge'] > 0][['BOX_ID','brand']].drop_duplicates()
    CC_costs=pd.merge(box_with_positive_charges, CC, on='BOX_ID',how='left').groupby('brand').agg({'BOX_ID': 'count',
    'TOTAL_COMPENSATION_AMOUNT': 'sum'}).reset_index().rename(columns={'BOX_ID': 'box_count'})
    CC_costs['weekly compensation'] = round(CC_costs['TOTAL_COMPENSATION_AMOUNT'] / Nb_weeks)
    return CC_costs

def get_CC_costs_brand(OTP,CC):
    BOX_Ids = OTP[['BOX_ID','brand']].drop_duplicates()
    CC_costs=pd.merge(BOX_Ids, CC, on='BOX_ID',how='left').groupby('brand').agg({'BOX_ID': 'count',
    'TOTAL_COMPENSATION_AMOUNT': 'sum'}).reset_index().rename(columns={'BOX_ID': 'box_count'})
    CC_costs['weekly compensation'] = round(CC_costs['TOTAL_COMPENSATION_AMOUNT'] / Nb_weeks)
    CC_costs['avg box comp'] = round(CC_costs['TOTAL_COMPENSATION_AMOUNT'] / CC_costs['box_count'])
    return CC_costs

def get_CC_costs(OTP,CC):
    # Does NOT make sense to look at the OTP level since duplicated box_id with different OTP_GROUP (du_id level)
    BOX_Ids = OTP['BOX_ID'].drop_duplicates()
    CC_costs=pd.merge(BOX_Ids, CC, on='BOX_ID',how='left')

    weekly_count = CC_costs['BOX_ID'].count()/Nb_weeks
    weekly_sum = round(CC_costs['TOTAL_COMPENSATION_AMOUNT'].sum()/Nb_weeks)
    avg_comp=round(weekly_sum/weekly_count,2)

    #print("Weekly CC compensation : $",weekly_sum)
    print("Avg comp per box shipped : $",avg_comp)
    return 

def get_positive_CC_costs(OTP,CC): # Only counting boxes with compensation
    BOX_Ids = OTP['BOX_ID'].drop_duplicates()
    CC_costs=pd.merge(BOX_Ids, CC, on='BOX_ID',how='left')
    CC_costs=CC_costs[CC_costs['TOTAL_COMPENSATION_AMOUNT']>0]
    
    weekly_count = CC_costs['BOX_ID'].count()/Nb_weeks
    weekly_sum = round(CC_costs['TOTAL_COMPENSATION_AMOUNT'].sum()/Nb_weeks)
    avg_comp=round(weekly_sum/weekly_count,2)

    print("Weekly CC compensation : $",weekly_sum)
    print("Avg comp per box compensated: $",avg_comp)
    return 

def get_CC_costs_for_LU(OTP,CC):
    BOX_Ids_LU = OTP[OTP['OTP_GROUP'].isin(['Late_1','Late_2','Undelivered'])]['BOX_ID'].drop_duplicates()
    CC_costs_LU=pd.merge(BOX_Ids_LU, CC, on='BOX_ID',how='left')

    weekly_count = len(BOX_Ids_LU)/Nb_weeks
    weekly_sum = round(CC_costs_LU['TOTAL_COMPENSATION_AMOUNT'].sum()/Nb_weeks)
    avg_comp=round(weekly_sum/weekly_count,2)

    print("Weekly CC compensation for LU: $",weekly_sum)
    print("nb weekly boxes LU: ",weekly_count)

    print("Avg comp per box LU : $",avg_comp)
    return 

def get_positive_CC_costs_for_LU(OTP,CC):
    BOX_Ids_LU = OTP[OTP['OTP_GROUP'].isin(['Late_1','Late_2','Undelivered'])]['BOX_ID'].drop_duplicates()
    CC_costs_LU=pd.merge(BOX_Ids_LU, CC, on='BOX_ID',how='left')
    CC_costs_LU=CC_costs_LU[CC_costs_LU['TOTAL_COMPENSATION_AMOUNT']>0]

    weekly_count = CC_costs_LU['BOX_ID'].count()/Nb_weeks
    weekly_sum = round(CC_costs_LU['TOTAL_COMPENSATION_AMOUNT'].sum()/Nb_weeks)
    avg_comp=round(weekly_sum/weekly_count,2)

    #print("Weekly CC compensation for LU: $",weekly_sum)
    print("nb weekly boxes LU compensated: ",weekly_count)
    print("Avg comp per box LU compensated: $",avg_comp)
    return 

Data preprocessing & Join OTP and Spends

In [5]:
OTP.rename(columns={'DU_ID': 'du_id','BRAND': 'brand'}, inplace=True)
OTP['Box_type'] = OTP['du_id'].apply(get_box_type)

Spend=Spend.rename(columns={'Address Correction Approved Amount':'correction_charge','Tracking Number':'TRACKING_CODE'})
Spend['correction_charge'] = Spend['correction_charge'].replace('[\$,]', '', regex=True).astype(float)

OTP_charges=pd.merge(OTP, Spend[['correction_charge','TRACKING_CODE']], on='TRACKING_CODE',how='left').groupby(['du_id','BOX_ID','brand','HELLOFRESH_WEEK','Box_type','OTP_GROUP','FINAL_ROOT_CAUSE']).agg({
    'correction_charge': 'sum'}).reset_index()

CC=CC.groupby('BOX_ID').agg({'TOTAL_COMPENSATION_AMOUNT':'sum'}).reset_index()

In [6]:
BOX_Ids_not_LU = OTP[~OTP['OTP_GROUP'].isin(['Late_1','Late_2','Undelivered'])]['BOX_ID'].drop_duplicates()
CC_costs_not_LU=pd.merge(BOX_Ids_not_LU, CC, on='BOX_ID',how='left')
CC_costs_not_LU=CC_costs_not_LU[CC_costs_not_LU['TOTAL_COMPENSATION_AMOUNT']>0]
OTP_with_CC_not_LU = pd.merge(CC_costs_not_LU,OTP,on='BOX_ID',how='left')
display(OTP_with_CC_not_LU)

,BOX_ID,TOTAL_COMPENSATION_AMOUNT,HELLOFRESH_WEEK,brand,du_id,OTP_GROUP,FINAL_ROOT_CAUSE,TRACKING_CODE,Box_type
0,CG11957124,9.60,2024-W02,GC,C119571240,On_Time,Weather,1ZT14T91P202323096,Original
1,US316191751,120.87,2024-W02,HF,H3161917510,On_Time,Undetermined,1Z934W3FP201780537,Original
2,US316201000,4.96,2024-W02,HF,H3162010000,On_Time,Undetermined,1Z6W7887P211204896,Original
3,US316160385,10.00,2024-W02,HF,H3161603850,On_Time,Undetermined,1Z0X6337P200797786,Original
4,US316160943,49.76,2024-W02,HF,H3161609430,On_Time,Undetermined,1Z2E2R79P220678840,Original
...,...,...,...,...,...,...,...,...,...
995615,FJ34449471,17.99,2023-W50,FA,F2985212410,On_Time,Undetermined,1ZY6E891P202701695,Original
995616,FJ34276641,27.98,2023-W50,FA,F2431831350,On_Time,Production Issues,HFDN0016627462,Original
995617,FJ34433609,22.00,2023-W50,FA,F2487182410,On_Time,Final Mile,SCD0000F00200450062,Original
995618,FJ34433609,22.00,2023-W50,FA,F2487182410,On_Time,Final Mile,SCD0000F00300996885,Original


Define Address flags boxes and Non-Address flags

In [7]:
address_flags = OTP_charges[(OTP_charges['FINAL_ROOT_CAUSE'].isin(('Access Issues', 'Address Issues'))) | (OTP_charges['correction_charge'] > 0)]
non_address_flags = OTP_charges[~(OTP_charges['FINAL_ROOT_CAUSE'].isin(('Access Issues', 'Address Issues'))) & ~(OTP_charges['correction_charge'] > 0)]

1) Get Avg Count and Percentage of du_ids with Address Flags, overall and then per brands

In [17]:
print("Total Nb of du_ids :",len(OTP))
#display(OTP.groupby('brand').size().reset_index())
print("Weekly Nb of du_ids with address flags :",len(address_flags)/Nb_weeks,"\t",round(len(address_flags)/len(OTP)*100,2),"%")
print("Weekly Nb of box_ids with address flags:", len(address_flags['BOX_ID'].drop_duplicates()) / Nb_weeks, "\t", round(len(address_flags['BOX_ID'].drop_duplicates()) / len(OTP['BOX_ID'].drop_duplicates()) * 100, 2), "%")
address_flags_per_brand=address_flags.groupby('brand').size().reset_index().rename(columns={0:'total'})
address_flags_per_brand['weekly avg']=address_flags_per_brand['total']/Nb_weeks
display(address_flags_per_brand)

Total Nb of du_ids : 11443115
Weekly Nb of du_ids with address flags : 4391.6 	 0.38 %
Weekly Nb of box_ids with address flags: 4218.1 	 0.38 %


,brand,total,weekly avg
0,EP,5368,536.8
1,FA,9777,977.7
2,GC,2428,242.8
3,HF,26343,2634.3


2. Get Avg Cound and Percentage of OTP group among du_ids with address flags

In [9]:
address_flags_OTP=get_OTP_Counts(address_flags)
#display(address_flags_OTP[['OTP_GROUP', 'weekly count', 'perc']])
flags_count=retrieve_OTP_weekly_counts(address_flags_OTP)
flags_perc=retrieve_OTP_perc(address_flags_OTP)
print(flags_count)
print(flags_perc)

{'L1': 1772.0, 'L2': 387.0, 'U': 1032.0}
{'L1': 0.404, 'L2': 0.088, 'U': 0.235}


3. Compare weekly compensation and avg comp per box shipped and compensated

In [21]:
print("For Address Flags boxes")
get_positive_CC_costs(address_flags,CC)
get_CC_costs(address_flags,CC)
get_CC_costs_for_LU(address_flags,CC)
get_positive_CC_costs_for_LU(address_flags,CC)

print("\n For Non Address Flags boxes")
get_positive_CC_costs(non_address_flags,CC) 
get_CC_costs(non_address_flags,CC)
get_CC_costs_for_LU(non_address_flags,CC)
get_positive_CC_costs_for_LU(non_address_flags,CC)


For Address Flags boxes
Weekly CC compensation : $ 136143
Avg comp per box compensated: $ 83.54
Avg comp per box shipped : $ 32.28
Weekly CC compensation for LU:  127752
nb weekly boxes LU:  3052.1
Avg comp per box LU : $ 41.86
nb weekly boxes LU compensated:  1473.3
Avg comp per box LU compensated: $ 86.71

 For Non Address Flags boxes
Weekly CC compensation : $ 5235622
Avg comp per box compensated: $ 42.28
Avg comp per box shipped : $ 4.7
Weekly CC compensation for LU:  2065139
nb weekly boxes LU:  77808.3
Avg comp per box LU : $ 26.54
nb weekly boxes LU compensated:  29863.8
Avg comp per box LU compensated: $ 69.15


In [13]:
41.9*3052

127878.8

4. Avg comp per brand

In [22]:
display(get_CC_costs_brand(address_flags,CC))
#print("Total : $",sum(get_CC_costs_brand(address_flags,CC)['weekly compensation'])," charged weekly in CC comp")

,brand,box_count,TOTAL_COMPENSATION_AMOUNT,weekly compensation,avg box comp
0,EP,4776,160084.36,16008.0,34.0
1,FA,9478,351852.40,35185.0,37.0
2,GC,2157,122442.96,12244.0,57.0
3,HF,25770,727051.91,72705.0,28.0


In [23]:
display(get_CC_costs_brand(non_address_flags,CC))

,brand,box_count,TOTAL_COMPENSATION_AMOUNT,weekly compensation,avg box comp
0,EP,1067787,2801644.61,280164.0,3.0
1,FA,3212106,32264959.93,3226496.0,10.0
2,GC,452099,1760800.45,176080.0,4.0
3,HF,6405136,15528810.94,1552881.0,2.0


5. Get Avg weekly carrier charges

In [24]:
display(get_carrier_charges(address_flags))
print("Total : $",sum(get_carrier_charges(address_flags)['weekly charges'])," charged weekly in carrier charges")

,brand,correction_charge,weekly charges
0,EP,26113.0,2611.0
1,FA,39143.0,3914.0
2,GC,11558.5,1156.0
3,HF,129910.0,12991.0


Total : $ 20672.0  charged weekly in carrier charges


6. Analysis of carrier charges vs OTP

In [25]:
charges_and_OTP=get_positive_carrier_charges_and_OTP(address_flags)
display(charges_and_OTP)
print("Avg fee :",round(sum(charges_and_OTP['weekly charges']/sum(charges_and_OTP['weekly count'])),2))
print("Total boxes with carrier charges :", sum(charges_and_OTP['weekly count']))
print("Total boxes with carrier charges and L/U:", sum(charges_and_OTP[charges_and_OTP['OTP_GROUP'].isin(['Late_1', 'Late_2', 'Undelivered'])]['weekly count']))

,OTP_GROUP,correction_charge,BOX_ID,weekly charges,weekly count,count perc
0,Early -1,1651.0,93,165.0,9.0,0.73
1,Early -2,69.0,4,7.0,0.0,0.00
2,Late_1,59018.0,3462,5902.0,346.0,28.18
3,Late_2,21559.0,1180,2156.0,118.0,9.61
4,On_Time,113102.5,6866,11310.0,687.0,55.94
5,Undelivered,11325.0,677,1132.0,68.0,5.54


Avg fee : 16.83
Total boxes with carrier charges : 1228.0
Total boxes with carrier charges and L/U: 532.0


7. Get weekly CC compensation for boxes with carrier charges

In [26]:
display(get_CC_for_corrected_boxes(address_flags,CC))
print("Weekly CC for boxes with carrier charges: $",sum(get_CC_for_corrected_boxes(address_flags,CC)['weekly compensation']))

,brand,box_count,TOTAL_COMPENSATION_AMOUNT,weekly compensation
0,EP,1420,25236.42,2524.0
1,FA,2161,51967.57,5197.0
2,GC,635,19349.45,1935.0
3,HF,7663,128910.64,12891.0


Weekly CC for boxes with carrier charges: $ 22547.0


In [37]:
0.0038*0.727*0.91

0.0025139660000000003